# Library

In [ ]:
import numpy as np 
import pandas as pd 
import os
from IPython.display import Markdown
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import folium
%matplotlib inline
import seaborn as sns
sns.set()
import pycountry
from plotly.offline import init_notebook_mode, iplot 
import plotly.offline as py
import plotly.express as ex
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
py.init_notebook_mode(connected=True)
import folium 
from folium import plugins
plt.style.use("seaborn-talk")
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['image.cmap'] = 'viridis'
from fbprophet import Prophet
pd.set_option('display.max_rows', None)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Uploading Data

In [ ]:
data = pd.read_csv('../input/covid19-india-cleaned-data/complete.csv')
covid_19_India = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
df = pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv', parse_dates=['Date'])

# Data Cleaning

In [ ]:
data.head()

In [ ]:
data['Total Cases'] = data['Total Confirmed cases (Indian National)'] + data['Total Confirmed cases ( Foreign National )'] 
data['Active Cases'] = data['Total Cases'] - (data['Cured/Discharged/Migrated'] + data['Death'])

In [ ]:
covid_19_India.head()

In [ ]:
covid_19_India['Confirmed'] = covid_19_India['ConfirmedIndianNational']+covid_19_India['ConfirmedForeignNational']
covid_19_India['Active']= covid_19_India['Confirmed']- (covid_19_India['Cured'] + covid_19_India['Deaths'])
covid_19_India

In [ ]:
date_wise_data = covid_19_India[["Date","Confirmed","Deaths","Cured",'Active']]
date_wise_data['Date'] = date_wise_data['Date'].apply(pd.to_datetime, dayfirst=True)
date_wise_data = date_wise_data.groupby(["Date"]).sum().reset_index()
def formatted_text(string):
    display(Markdown(string))
date_wise_data.head()

In [ ]:
df.head()

In [ ]:
df['Name of State / UT'] = df['Name of State / UT'].str.replace('Union Territory of ', '')
for i in ['Total Confirmed cases (Indian National)', 'Total Confirmed cases ( Foreign National )', 'Cured/Discharged/Migrated', 'Death']:
    df[i] = df[i].astype('int')    
df['Total cases'] = df['Total Confirmed cases (Indian National)'] + df['Total Confirmed cases ( Foreign National )']
df['Moratlity rate'] = df['Death']/df['Total cases']
df['Recovery rate'] = df['Cured/Discharged/Migrated']/df['Total cases']
df['Active'] = df['Total cases'] - df['Death'] - df['Cured/Discharged/Migrated']
df.columns = ['Date', 'State/UT', 'Confirmed(Indian)', 'Confirmed(Foreigner)', 'Cured/Discharged/Migrated', 'Latitude', 'Longitude', 'Death', 'Total cases', 'Mortality rate', 'Recovery rate', 'Active']
df = df[['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed(Indian)', 'Confirmed(Foreigner)', 'Total cases', 'Active', 'Death', 'Cured/Discharged/Migrated', 'Mortality rate', 'Recovery rate']]
df.head()

In [ ]:
latest = df[df['Date']==max(df['Date'])]
latest_day = max(df['Date'])
day_before = latest_day - timedelta(days = 1)
latest_day_df = df[df['Date']==latest_day].set_index('State/UT')
day_before_df = df[df['Date']==day_before].set_index('State/UT')
temp = pd.merge(left = latest_day_df, right = day_before_df, on='State/UT', suffixes=('_lat', '_bfr'), how='outer')
latest_day_df['New cases'] = temp['Total cases_lat'] - temp['Total cases_bfr']
latest = latest_day_df.reset_index()
latest.fillna(1, inplace=True)

# **Overall view of the situation**

In [ ]:
#Overall 
ac= data['Active Cases'].sum()
rvd = data['Cured/Discharged/Migrated'].sum()
dth = data['Death'].sum()
fig = go.Figure(data=[go.Pie(labels=['Active Cases','Cured','Death'],
                             values= [ac,rvd,dth],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#2fcc41','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.show()

# **Trend of Weekly COVID-19 cases in India**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Confirmed'],
                    mode='lines+markers',marker_color='blue',name='Total Cases'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'],y=date_wise_data['Active'], 
                mode='lines+markers',marker_color='purple',name='Active'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Cured'],
                mode='lines+markers',marker_color='green',name='Recovered'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Deaths'], 
                mode='lines+markers',marker_color='red',name='Deaths'))
fig.update_layout(title_text='Trend of Weekly Coronavirus Cases in India',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
cnf = '#263fa3' # confirmed - blue
act = '#fe9801' # active case - yellow
rec = '#21bf73' # recovered - green
dth = '#de260d' # death - red
tmp = date_wise_data.melt(id_vars="Date",value_vars=['Deaths','Cured' ,'Active','Confirmed'],
                 var_name='Case',value_name='Count')
fig = px.area(tmp, x="Date", y="Count",color='Case',title='Trend Over Weeks',color_discrete_sequence = [dth,rec,act,cnf])
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
df_india_data = covid_19_India[['Date', 'State/UnionTerritory','Confirmed','ConfirmedIndianNational','ConfirmedForeignNational','Cured','Deaths','Active']]

In [ ]:
spread = df_india_data.groupby(['Date', 'State/UnionTerritory'])['Confirmed'].sum().reset_index().sort_values('Confirmed', ascending=True)
fig = px.area(spread, x="Date", y="Confirmed",color='State/UnionTerritory',title='State Wise Spread over time',height=700,
              color_discrete_sequence = ex.colors.cyclical.Edge)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')

In [ ]:
fig = px.colors.sequential.Inferno


In [ ]:
temp = latest.sort_values('Total cases', ascending=True)
fig = go.Figure(data=[
    go.Bar(name='Active', y=temp['State/UT'], x=temp['Active'], orientation='h'),
    go.Bar(name='Death', y=temp['State/UT'], x=temp['Death'], orientation='h'),
    go.Bar(name='Cured/Discharged/Migrated', y=temp['State/UT'], x=temp['Cured/Discharged/Migrated'], orientation='h')])
# Change the bar mode
fig.update_layout(barmode='stack', height=900)
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

# **Covid-19 State Wise insight**

In [ ]:
temp = latest.sort_values('Total cases', ascending=False)
state_order = temp['State/UT']
fig = px.bar(temp,x="Total cases", y="State/UT", color='State/UT', title='State Wise Confirmed Cases', orientation='h', text='Total cases', 
             height=900,color_discrete_sequence = px.colors.sequential.Plasma_r)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
fig = px.treemap(latest, path=["State/UT"], values="Cured/Discharged/Migrated" , height=500,
                 title='State Wise Number of Cured Cases', 
                 color_discrete_sequence = ex.colors.cyclical.Twilight)
fig.data[0].textinfo = 'label+text+value'
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
fig = px.treemap(latest, path=["State/UT"], values="Death", height=500,
                 title='State Wise Number of Death', color_discrete_sequence = ex.colors.cyclical.Twilight)
fig.data[0].textinfo = 'label+text+value'
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
covid_19_India.head()

# Over the time 

In [ ]:
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

tmp = df.copy()
tmp['Date'] = tmp['Date'].dt.strftime('%Y/%m/%d')
fig = px.scatter_geo(tmp, lat="Latitude", lon="Longitude", color='Total cases', size='Total cases', projection="natural earth",
                     hover_name="State/UT", scope='asia', animation_frame="Date",
                     color_continuous_scale=px.colors.diverging.curl,center={'lat':20, 'lon':78}, 
                     range_color=[0, max(tmp['Total cases'])])
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

**Note: This work is highly inspired from few other kaggle kernels , github sources and other data science resources. Any traces of replications, which may appear , is purely co-incidental. Due respect & credit to all my fellow kagglers.**